In [ ]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
from scipy.signal import savgol_filter
from scipy.stats import linregress
from scipy.optimize import least_squares

##### Mediciones usadas
Salvo en el caso de electrodo largo con campo se consideraron únicamente las mediciones de oscilación completa

In [ ]:
data_oscillations = {
        'Corto sin campo':{'Mediciones 11-2 - Lara\\1':[2,5,6,7,8,9,10],
                           'Mediciones 11-2 - Lara\\2':[2,3,4,7,10],
                           'Mediciones 11-2 - Lara\\3':[1,4,5,9],
                           'Mediciones 11-2 - Lara\\4':[1,5,6],
                           'Mediciones 11-2 - Lara\\5':[2,7,10],
                           'Mediciones 11-2 - Lara\\6':[5,6,7,9,10],
                           'Mediciones 11-2 - Lara\\7':[5,8,9,10],
                           'Mediciones 11-2 - Lara\\8':[1,2,3,4,5],
                           'Mediciones 11-2 - Lara\\9':[5,9],
                           'Mediciones 11-2 - Lara\\10':[1,5]},
        'Corto con campo':{'Mediciones con campo 18-2 - Agus\\1':[2,3,4,5,6,7,8,11,12,13,14,15,18,19,21,23,24,25,26,27,28,29,30,31,33,34,35,36,37,38,40,41,42,43,44,45,46,47,48,49]},
        'Largo sin campo':{'Mediciones 22-2 - Agus\\1':[2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,21,22,23,24,25,26,27,28,29,32,33,35,39,40,42,43,44,45,47,48,49],
                           'Mediciones 22-2 - Agus\\2':[2,5,6,7,9,10,11,12,13,15,16,17,18,27,30,31,32,33,34,35,37,38,41,42,43,44,45,50],
                           'Mediciones 22-2 - Agus\\3':[1,2,3,4,6,7,8,9,10,11,14,15,16,17,21,23,25,26,27,29,30,31,34,36,39,42,44,46,47,49]},
        'Largo con campo':{'Mediciones con campo 23-2 - Lara\\1':[2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21],
                           'Mediciones con campo 23-2 - Lara\\3':[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17],
                           'Mediciones con campo 23-2 - Lara\\4':[1,2,5,6,7,8,11,12,13,14,15,16,17,19,20,21,22,23,24]}
        }



def cant_config(config):
    cant = 0
    for day in data_oscillations[config]:
        cant += len(data_oscillations[config][day])
    return cant

corto_con = cant_config('Corto con campo')
largo_con = cant_config('Largo con campo')
corto_sin = cant_config('Corto sin campo')
largo_sin = cant_config('Largo sin campo')

print('Mediciones analizadas por configuracion')
print('Con campo: ',corto_con+largo_con)
print('Sin campo: ',corto_sin+largo_sin)


##### Funciones para analizar mediciones y realizar fiteos

In [ ]:
def get_cut_signal(time, voltage, capacitor, field = False):     

    if field:
        I_limite=1.31e-7
    else:
        I_limite=3.4e-8
    P=30
    
    #integration
    filtered_voltage = savgol_filter(voltage, 15, 2)
    integration = np.array([np.trapz(filtered_voltage[:l],time[:l]) for l in range(len(voltage))])
    
    linear_time = []
    linear_intvoltage = []
    for l in range(len(time)):
        if time[l]<=float(min(time))+0.5*10**(-5):
            linear_time.append(time[l])
            linear_intvoltage.append(integration[l])
    
    slope, intercept, r_value, p_value, std_err = linregress(linear_time,linear_intvoltage)
    background_adjustment = slope*np.array(time)+intercept
    backgroundless_current = integration-background_adjustment
      
    k=0
    time_fit=[]
    integration_fit=[]
    for l in range(len(time)):
        if backgroundless_current[l]>=I_limite and k==0:
            time_fit=time[(l-P):]
            integration_fit=backgroundless_current[(l-P):]
            capacitor_fit=capacitor[(l-P):]
            k=1
    
    return time_fit,filtered_voltage,background_adjustment,backgroundless_current,integration_fit,np.abs(capacitor_fit[0])


def get_guess_parameters(time, backgroundless_current):
    k=0
    time_fit=[]
    integration_fit=[]
    for i in range(len(time)):
        if backgroundless_current[i]>=I_limite and k==0:
            T1=time[i-P]
            time_fit=time[(i-P):]
            integration_fit=backgroundless_current[(i-P):]
            k=1

    k=0
    if integration_fit[len(integration_fit)-300]>(-4e-8):#*C_rog):
        for i in range(len(time_fit)):
            if k==0 and time_fit[i]>(T1+1.53*10**(-5)):
                index=list(range(i,len(time_fit)))
                time_fit=np.delete(time_fit,index)
                integration_fit=np.delete(integration_fit,index)
                T2= (time_fit[len(time_fit)-1]-T1)*2
                k=1
    elif integration_fit[len(integration_fit)-300]<=(-4e-8):#*C_rog):
        T2=time_fit[len(time_fit)-1]-T1

    guess_amplitude=max(integration_fit)
    guess_R=0.3
    guess_L=0.000002
    guess_omega = 2*np.pi/T2 
    guess_alpha = guess_R/(2*guess_L)
    guess_t0 = T1
    guess_parameters = [guess_amplitude, guess_omega, guess_alpha, guess_t0]
    return guess_parameters

CRog = 1.2207643130e9
def cos_fit_fun_damped(parameters,time):
    a = parameters[0]
    omega = parameters[1]
    alpha = parameters[2]
    t0 = parameters[3]
    y = a * np.sin(omega * (time-t0) ) * np.exp(-alpha*(time-t0))
    y = y*CRog
    return y

def get_residuals(parameters, position_data, time_data):
    theoretical_function = cos_fit_fun_damped(parameters,time_data)
    residuals = np.abs(theoretical_function - position_data)
    return residuals

def get_fit(time, backgroundless_current):
    guess_parameters = get_guess_parameters(time, backgroundless_current)
    res_lsq = least_squares(get_residuals, guess_parameters, args=(integration_fit,time_fit))
    best_parameters = res_lsq['x']
    fitted_function = cos_fit_fun_damped(best_parameters, time_fit)
    return fitted_function

# defining function to perform all calculations
def get_RLC_params(time, backgroundless_current):
    guess_parameters = get_guess_parameters(time, backgroundless_current)
    res_lsq = least_squares(get_residuals, guess_parameters, args=(integration_fit,time_fit))
    best_parameters = res_lsq['x']
    pcov = calcular_cov(res_lsq,y)
    pstd = np.sqrt(np.diag(pcov))
    Amplitude = [round(best_parameters[0],3),round(pstd[0],3)]
    Omega = [round(best_parameters[1],3),round(pstd[1],3)]
    Alpha = [round(best_parameters[2],3),round(pstd[2],3)]
    C=9.6*10**(-6)
    W02=(best_parameters[1]**2+best_parameters[2]**2)
    L=1/(C*W02)
    deltaW02=2*best_parameters[1]*pstd[1]+2*best_parameters[2]*pstd[2]
    deltaL=(deltaW02/W02)*L
    R=best_parameters[2]*2*L
    deltaR=(pstd[2]/best_parameters[2]+deltaL/L)*R
    Resistance = [round(R,7),round(deltaR,7)]
    Inductance = [round(L,10),round(deltaL,10)]
    Omega0 = [round(W02,1),round(deltaW02,1)]
    Voltage = best_parameters[0]*best_parameters[1]*L
    #each parameter returned is a list shaped [parameter, parameter error]
    return Amplitude, Omega, Omega0, Alpha, Resistance, Inductance, Voltage

##### Gráfico de corrientes filtradas

In [ ]:
carpeta='C:\\Users\\DG\\Desktop\\Labo 7 Pereyra Grau y Schvartzman\\Mediciones con campo 18-2 - Agus\\'

ch1 = pd.read_csv(carpeta+'Mediciones_ch1_1.csv', engine='python', index_col = 0) #capacitor
ch2 = pd.read_csv(carpeta+'Mediciones_ch2_1.csv', engine='python', index_col = 0) #rogowski

time = np.array(ch1.loc[0]).astype(np.float)
voltage = np.array(ch2.loc[1]).astype(np.float)
V_capacitor= np.array(ch1.loc[1]).astype(np.float)

time, filtered_voltage, background_adjustment, backgroundless_current, integration, V0 = get_cut_signal(time, voltage, V_capacitor, True)

SMALL_SIZE = 12
MEDIUM_SIZE = 22
BIGGER_SIZE = 50

plt.rc('font', size=SMALL_SIZE)
plt.rc('axes', titlesize=SMALL_SIZE)
plt.rc('axes', labelsize=SMALL_SIZE)
plt.rc('xtick', labelsize=SMALL_SIZE)
plt.rc('ytick', labelsize=SMALL_SIZE)
plt.rc('legend', fontsize=SMALL_SIZE)
plt.rc('figure', titlesize=BIGGER_SIZE)

fig, axs = plt.subplots(2, 1, constrained_layout=True,figsize=(10,10))
axs[0].plot(time*1000000, voltage, label = 'Voltaje Rogowski')
axs[0].plot(time*1000000, filtered_voltage, label = 'Señal filtrada')
axs[0].set_ylabel(r'Voltaje [V]', fontsize=13)
axs[0].set_xlabel(r'Tiempo [$\mu s$]', fontsize=13)
axs[0].tick_params(axis='both', labelsize=13)
axs[0].legend()
axs[0].grid()
axs[1].plot(time*1000000, integration*1000000, label = 'Voltaje integrado')
axs[1].plot(time*1000000, background_adjustment*1000000, label = 'Ajuste del fondo')
axs[1].plot(time*1000000, backgroundless_current*1000000, label = 'Corriente sin el fondo')
axs[1].set_ylabel(r'Corriente [$\mu A$]', fontsize=13)
axs[1].set_xlabel(r'Tiempo [$\mu s$]', fontsize=13)
axs[1].tick_params(axis='both', labelsize=13)
axs[1].legend()
axs[1].grid()

##### Gráfico de ajuste

In [ ]:
fitted_function = get_fit(time, backgroundless_current)
plt.figure(2)
plt.plot(np.array(time)*1000000, backgroundless_current*1000000,  label='Corriente obtenida')
plt.plot(np.array(time_fit)*1000000, fitted_function*1000000, color = 'red', linewidth = 2.0, label='Ajuste')
plt.xlabel(r"Tiempo [$\mu s$]")
plt.ylabel("Corriente [$\mu A$]")
plt.legend()
plt.grid()
plt.show()

##### Cálculo de C Rogowski y $V_0$

In [ ]:
def get_c_rogowski(time,current):
    T1=time[0]    
    #guess parameters
    T2= time[len(time)-1]-T1
    guess_amplitude=max(current)
    guess_R=0.3
    guess_L=0.000002
    guess_omega = np.pi/(T2-T1) 
    guess_alpha = guess_R/(2*guess_L)
    guess_t0 = T1
    guess_parameters = [guess_amplitude, guess_omega, guess_alpha, guess_t0]
    
    #performing fit
    res_lsq = least_squares(get_residuals, guess_parameters, args=(current,time))
    best_parameters = res_lsq['x']
    fitted_function = cos_fit_fun_damped(best_parameters, time)
    
    #calculating error
    pcov = calcular_cov(res_lsq,current)
    pstd = np.sqrt(np.diag(pcov))
        
    #defining parameters
    Amplitude = best_parameters[0]
    Omega = best_parameters[1]
    Alpha = best_parameters[2]
    C=9.6*10**(-6)
    W02=(best_parameters[1]**2+best_parameters[2]**2)
    L=1/(C*W02)
    deltaW02=2*best_parameters[1]*pstd[1]+2*best_parameters[2]*pstd[2]
    deltaL=(deltaW02/W02)*L
    R=best_parameters[2]*2*L
    deltaR=(pstd[2]/best_parameters[2]+deltaL/L)*R
    
    F=0
    for l in range(len(time)):
        if time[l]>=best_parameters[3] and current[0]!=1 and F==0:
            V0=-V_capacitor[l]
            F=1
        elif time[l]>=best_parameters[3] and current[0]==1 and F==0:
            V0=0
            F=1
    
    #defining C Rogowski
    C_Rog = V0/(Amplitude*Omega*L)
    return C_Rog

directory='C:\\Users\\DG\\Desktop\\Labo 7 Pereyra Grau y Schvartzman\\'

c = []
V0 = []
for data_type in data_oscillations:
    if data_type.startswith('Corto con') or data_type.startswith('Largo con'):
        field = True
    else:
        field = False
    for data in data_oscillations[data_type]:
        folder = data.split('\\')
        number = folder[1]
        folder = folder[0]+'\\'
        
        dir_ch1 = directory+folder+'Mediciones_ch1_{}.csv'.format(number)
        dir_ch2 = directory+folder+'Mediciones_ch2_{}.csv'.format(number)
        ch1 = pd.read_csv(dir_ch1, engine='python', index_col = 0) #capacitor
        ch2 = pd.read_csv(dir_ch2, engine='python', index_col = 0) #rogowski
        time = np.array(ch1.loc[0]).astype(np.float)
        voltage = np.array(ch2.loc[int(number)]).astype(np.float)
        V_capacitor= np.array(ch1.loc[int(number)]).astype(np.float)
        
        time, filtered_voltage, background_adjustment, backgroundless_current, integration, V0 = get_cut_signal(time, voltage, V_capacitor, field)
        c.append(get_c_rogowski(time,integration))
        V0.append(V0)

##### Cálculo de R y L

In [ ]:
def get_R_and_L(time, voltage, V_capacitor, field):
    time, filtered_voltage, background_adjustment, backgroundless_current, integration, V0 = get_cut_signal(time, voltage, V_capacitor, field)
    Amplitude, Omega, Omega0, Alpha, Resistance, Inductance, Voltage = get_RLC_params(time, backgroundless_current)
    return Resistance[0], Inductance[0]

R = []
L = []
for data_type in data_oscillations:
    if data_type.startswith('Corto con') or data_type.startswith('Largo con'):
        field = True
    else:
        field = False
    for data in data_oscillations[data_type]:
        folder = data.split('\\')
        number = folder[1]
        folder = folder[0]+'\\'
        
        dir_ch1 = directory+folder+'Mediciones_ch1_{}.csv'.format(number)
        dir_ch2 = directory+folder+'Mediciones_ch2_{}.csv'.format(number)
        ch1 = pd.read_csv(dir_ch1, engine='python', index_col = 0) #capacitor
        ch2 = pd.read_csv(dir_ch2, engine='python', index_col = 0) #rogowski
        time = np.array(ch1.loc[0]).astype(np.float)
        voltage = np.array(ch2.loc[int(number)]).astype(np.float)
        V_capacitor= np.array(ch1.loc[int(number)]).astype(np.float)
        
        r, l = get_R_and_L(time, voltage, V_capacitor, field)
        R.append(r)
        L.append(l)

##### Mediciones de período largo

In [ ]:
i = 1
j = 1
directory = 'Mediciones masa ablada Agus/Mediciones_ch1_'+str(i)+'_efec.csv'
ch1 = pd.read_csv(directory, engine='python', index_col = 0)
time = np.array(ch1.loc[0]).astype(float)
capacitor = np.array(ch1.loc[j]).astype(float)

plt.plot(time, capacitor, label='Capacitor')
plt.xlabel('Tiempo [s]')
plt.ylabel('Voltaje [V]')
plt.legend()
plt.savefig('Muestrapicos.png', dpi=500)

##### Cálculo de masa ablada

In [ ]:
def entorno_capacitor(indice):
    threshold = 500
    if indice < threshold:
        entorno = capacitor[:indice+threshold]
    elif indice > len(capacitor) - threshold:
        entorno = capacitor[indice-threshold:]
    else:
        entorno = capacitor[indice-threshold:indice+threshold]
    return entorno

def get_maxs(time, capacitor):
    time_max = []
    for i in range(len(capacitor)):
        entorno = entorno_capacitor(i)
        if all(capacitor[i] >= j for j in entorno) and capacitor[i] > -200:
            time_max.append(time[i])
    for i in range(len(time_max)):
        for i in range(len(time_max)):
            if i < len(time_max)-1:
                if np.abs(time_max[i]-time_max[i+1]) < 0.01:
                    med_time = (time_max[i]+time_max[i+1])/2
                    time_max[i] = med_time
                    del time_max[i+1]
    
    return time_max

def get_freq(time, capacitor):
    maxs = get_maxs(time, capacitor)
    if (len(maxs) == 1) or (len(maxs) == 0):
        return None
    freq = np.abs(maxs[0] - maxs[1])
    if len(maxs) == 3:
        freq1 = np.abs(maxs[1] - maxs[2])
        freq = (freq+freq1)/2
    return freq

def get_freq_stats(field=False):
    if field:
        up_to = 7
    else:
        up_to = 14
    frecuencias = []
    for i in range(1,up_to):
        if field:
            directory = 'Mediciones masa ablada con campo Lara/Mediciones_ch1_'+str(i)+'_efec.csv'
        else:
            directory = 'Mediciones masa ablada Agus/Mediciones_ch1_'+str(i)+'_efec.csv'
        ch1 = pd.read_csv(directory, engine='python', index_col = 0) #capacitor
        for j in range(1,11):
            time = np.array(ch1.loc[0]).astype(float)
            capacitor = np.array(ch1.loc[j]).astype(float)
            freq = get_freq(time, capacitor)
            if freq == None:
                pass
            elif freq > 2:
                print('Outlier en frecuencia: numero de medicion '+str(i)+', tira de medicion '+str(j))
            else:
                frecuencias.append(freq)
                
    return np.mean(frecuencias), np.std(frecuencias)

freq_sin, freq_sin_std = get_freq_stats()
print('Frecuencia de disparo sin campo: ',round(freq_sin,5),' +- ',round(freq_sin_std,5))

#freq_con, freq_con_std = get_freq_stats(True)
#print('Frecuencia de disparo con campo: ',freq_con,' +- ',freq_con_std)

##### Parámetros medios según configuración

In [ ]:
def get_parameters(data_config):
    if data_config == 'corto_sin':
        R = 0.388
        L = 2.78e-6
        V0 = 816
    elif data_config == 'corto_con':
        R = 0.426
        L = 2.43e-6
        V0 = 909
    elif data_config == 'largo_sin':
        R = 0.320
        L = 2.759e-6
        V0 = 853
    elif data_config == 'largo_con':
        R = 0.335
        L = 2.39e-6
        V0 = 855
        
    C = 9.6e-6
    alpha = R/(2*L)
    omega = np.sqrt(4*L/C-R**2)/(2*L)
    a = V0/(omega*L*CRog)
    t0 = 0
    return [a,omega,alpha,t0]

##### Funciones para plotear simulaciones

In [ ]:
def plot_time_profile(data_config):

    # dataframes
    perfilestemp = pd.read_csv("Simulaciones/perfiles_temp_{}.txt".format(data_config), sep = ' ', engine='python')  

    # PERFILES TEMP
    sim_time = []
    sim_current = []
    sim_Ibit = []
    sim_R = []
    sim_thrust = []
    sim_mass = []
    sim_lorentz = []

    for index, row in perfilestemp.iterrows():
        sim_time.append(row['Unnamed: 3'])
        if row['Unnamed: 4']<0 and index<=95:
            sim_current.append(row['Vcap'])
            sim_Ibit.append(row['Unnamed: 12'])
            sim_R.append(row['Unnamed: 8'])
            sim_thrust.append(row['Unnamed: 14'])
            sim_mass.append(row['res_plasma'])
            sim_lorentz.append(row['Unnamed: 20'])
        elif index>95:
            sim_current.append(row['Unnamed: 5'])
            sim_Ibit.append(row['Unnamed: 10'])
            sim_R.append(row['Unnamed: 7'])
            sim_thrust.append(row['Unnamed: 12'])
            sim_mass.append(row['Unnamed: 14'])
            sim_lorentz.append(row['Unnamed: 18'])
        else:
            sim_current.append(row['Unnamed: 7'])
            sim_Ibit.append(row['Unnamed: 13'])
            sim_R.append(row['Unnamed: 9'])
            sim_thrust.append(row['Unnamed: 15'])
            sim_mass.append(row['Unnamed: 17'])
            sim_lorentz.append(row['Velec'])

    mes_current = cos_fit_fun_damped(get_parameters(data_config),np.array(sim_time))
    # plot
    fig, axs = plt.subplots(5, 1, constrained_layout=True,figsize=(10,10))
    axs[0].plot(sim_time, sim_current, label = 'simulated data')
    axs[0].plot(sim_time, mes_current, label = 'measured data')
    axs[0].set_ylabel('Corriente [A]', fontsize=13)
    axs[0].set_xlabel('Tiempo [s]', fontsize=13)
    axs[0].tick_params(axis='both', labelsize=13)
    axs[0].legend()
    axs[1].plot(sim_time, sim_Ibit, label = 'Ibit')
    axs[1].set_ylabel('Impulso [Ns]', fontsize=13)
    axs[1].set_xlabel('Tiempo [s]', fontsize=13)
    axs[1].tick_params(axis='both', labelsize=13)
    axs[1].legend()
    axs[2].plot(sim_time, sim_R, label = 'Resistance')
    axs[2].set_ylabel('Resistencia [Ohm]', fontsize=13)
    axs[2].set_xlabel('Tiempo [s]', fontsize=13)
    axs[2].tick_params(axis='both', labelsize=13)
    axs[2].legend()
    axs[3].plot(sim_time, sim_thrust, label = 'Thrust')
    axs[3].plot(sim_time, sim_lorentz, label = 'Lorentz')
    axs[3].set_ylabel('Fuerza [N]', fontsize=13)
    axs[3].set_xlabel('Tiempo [s]', fontsize=13)
    axs[3].tick_params(axis='both', labelsize=13)
    axs[3].legend()
    axs[4].plot(sim_time, sim_mass, label = 'Mass')
    axs[4].set_ylabel('Masa [kg]', fontsize=13)
    axs[4].set_xlabel('Tiempo [s]', fontsize=13)
    axs[4].tick_params(axis='both', labelsize=13)
    axs[4].legend()
    fig.suptitle(data_config, fontsize=15)

def plot_x_profile(data_config):
    # perfil X
    perfiles1 = pd.read_csv("Simulaciones/perfiles1_{}.txt".format(data_config), header=1, sep = '  ', engine='python')

    x = list(perfiles1['x [m]'])
    rho = list(perfiles1[' rho [kg/m**3]'])
    u = list(perfiles1[' u [m/s]'])
    T = list(perfiles1[' T [eV]'])
    ne = list(perfiles1['ne[m**-3]'])

    fig, axs = plt.subplots(4, 1, constrained_layout=True,figsize=(10,10))
    axs[0].plot(x, u, label = 'u')
    axs[0].set_ylabel('Velocidad [m/s]', fontsize=13)
    axs[0].set_xlabel('Posición [m]', fontsize=13)
    axs[0].tick_params(axis='both', labelsize=13)
    axs[0].legend()
    axs[1].plot(x, T, label = 'T')
    axs[1].set_ylabel('Temperatura [eV]', fontsize=13)
    axs[1].set_xlabel('Posición [m]', fontsize=13)
    axs[1].tick_params(axis='both', labelsize=13)
    axs[1].legend()
    axs[2].plot(x, ne, label = 'ne')
    axs[2].set_ylabel('ne [1/m^3]', fontsize=13)
    axs[2].set_xlabel('Posición [m]', fontsize=13)
    axs[2].tick_params(axis='both', labelsize=13)
    axs[2].legend()
    axs[3].plot(x, rho, label = 'rho')
    axs[3].set_ylabel('rho [kg/m^3]', fontsize=13)
    axs[3].set_xlabel('Posición [m]', fontsize=13)
    axs[3].tick_params(axis='both', labelsize=13)
    axs[3].legend()
    fig.suptitle(data_config, fontsize=15)

##### Ploteos de las simulaciones

In [ ]:
data_config = 'corto_sin'

plot_time_profile(data_config)
plot_x_profile(data_config)

In [ ]:
data_config = 'corto_con'

plot_time_profile(data_config)
plot_x_profile(data_config)

In [ ]:
data_config = 'largo_sin'

plot_time_profile(data_config)
plot_x_profile(data_config)

In [ ]:
data_config = 'largo_con'

plot_time_profile(data_config)
plot_x_profile(data_config)